In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#import matplotlib.pyplot as plt
import psycopg2

In [2]:
fact_table_cols = [('Individual_Key', 'Resolved', 'Fatal', 'Special_Measure_Key', 'PHU_Key', 'Weather_Key', 'Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key', 'Specimen_Date_Key')]
individual_cols = [('surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city')]
fact_table = []
individual_table = []

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        return psycopg2.connect(
            host="www.eecs.uottawa.ca",
            database="group_21",
            user="****",
            password="****",
            port="15432"
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


connection = connect()
print('***Connection established***')

cur = connection.cursor()
        
# execute a statement
cur.execute('SELECT * FROM fact_table_v2')
rows = cur.fetchall()
fact_table = rows
print(fact_table[:10])


print("\n\n")


cur.execute('SELECT * FROM individuals')
rows = cur.fetchall()
individual_table = rows
print(individual_table[:10])

cur.close()
connection.close()
print('***Connection terminated***')

Connecting to the PostgreSQL database...
***Connection established***
[(250, 'yes', 'no', 11, 3895, 123, 27, 32, 32, 29), (322, 'yes', 'no', 11, 3895, 110, 16, 19, 19, 18), (326, 'no', 'yes', 11, 3895, 96, 1, 5, 28, 1), (347, 'yes', 'no', 11, 3895, 107, 9, 16, 10, 9), (692, 'yes', 'no', 11, 3895, 116, 29, 25, 25, 24), (868, 'yes', 'no', 11, 3895, 112, 23, 21, 21, 19), (1347, 'yes', 'no', 11, 3895, 104, 8, 13, 13, 11), (2760, 'yes', 'no', 11, 3895, 129, 23, 38, 38, 35), (3354, 'yes', 'no', 11, 3895, 123, 26, 32, 32, 30), (4742, 'yes', 'no', 2, 3895, 151, 45, 60, 60, 59)]



[(0, 259, datetime.date(2020, 3, 25), '50s', 'MALE', 'NO KNOWN EPI LINK', 'No', 'Toronto'), (1, 5981, datetime.date(2020, 3, 16), '20s', 'MALE', 'TRAVEL', 'No', 'Toronto'), (2, 7345, datetime.date(2020, 3, 23), '50s', 'MALE', 'OB', 'Yes', 'Toronto'), (3, 7354, datetime.date(2020, 3, 24), '20s', 'FEMALE', 'CC', 'No', 'Toronto'), (4, 8096, datetime.date(2020, 3, 28), '70s', 'FEMALE', 'TRAVEL', 'No', 'Ottawa'), (5, 8319

In [3]:
fact_df = pd.DataFrame(fact_table, columns=['Individual_Key', 'Resolved', 'Fatal', 'Special_Measure_Key', 'PHU_Key', 'Weather_Key', 'Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key', 'Specimen_Date_Key'])
fact_df.head()

,Individual_Key,Resolved,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Onset_Date_Key,Reported_Date_Key,Test_Date_Key,Specimen_Date_Key
0,250,yes,no,11,3895,123,27,32,32,29
1,322,yes,no,11,3895,110,16,19,19,18
2,326,no,yes,11,3895,96,1,5,28,1
3,347,yes,no,11,3895,107,9,16,10,9
4,692,yes,no,11,3895,116,29,25,25,24


### Label Encoding for Resolved

In [4]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
fact_df['Resolved_Cat'] = labelencoder.fit_transform(fact_df['Resolved'])
fact_df.head()

,Individual_Key,Resolved,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Onset_Date_Key,Reported_Date_Key,Test_Date_Key,Specimen_Date_Key,Resolved_Cat
0,250,yes,no,11,3895,123,27,32,32,29,1
1,322,yes,no,11,3895,110,16,19,19,18,1
2,326,no,yes,11,3895,96,1,5,28,1,0
3,347,yes,no,11,3895,107,9,16,10,9,1
4,692,yes,no,11,3895,116,29,25,25,24,1


### One Hot Encoding for Resolved

In [5]:
# creating instance of one-hot-encoder

enc = OneHotEncoder(handle_unknown='ignore')


In [6]:
# passing bridge-types-cat column (label encoded values of bridge_types)

enc_df = pd.DataFrame(enc.fit_transform(fact_df[['Resolved_Cat']]).toarray())


In [7]:
# merge with main df bridge_df on key values

fact_df = fact_df.join(enc_df)
fact_df.rename(columns={0: "Resolved_no", 1: "Resolved_yes"},inplace = True)
fact_df=fact_df.drop(['Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key','Specimen_Date_Key', 'Resolved_Cat','Resolved'], axis=1)
fact_df.head()

,Individual_Key,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Resolved_no,Resolved_yes
0,250,no,11,3895,123,0.0,1.0
1,322,no,11,3895,110,0.0,1.0
2,326,yes,11,3895,96,1.0,0.0
3,347,no,11,3895,107,0.0,1.0
4,692,no,11,3895,116,0.0,1.0


### Label Encoding for Fatal

In [8]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
fact_df['Fatal_Cat'] = labelencoder.fit_transform(fact_df['Fatal'])
fact_df.head()

,Individual_Key,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Resolved_no,Resolved_yes,Fatal_Cat
0,250,no,11,3895,123,0.0,1.0,0
1,322,no,11,3895,110,0.0,1.0,0
2,326,yes,11,3895,96,1.0,0.0,1
3,347,no,11,3895,107,0.0,1.0,0
4,692,no,11,3895,116,0.0,1.0,0


### One Hot Encoding for Fatal

In [9]:
# creating instance of one-hot-encoder

enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)

enc_df = pd.DataFrame(enc.fit_transform(fact_df[['Fatal_Cat']]).toarray())

# merge with main df bridge_df on key values

fact_df = fact_df.join(enc_df)
fact_df.rename(columns={0: "Fatal_no", 1: "Fatal_yes"},inplace = True)
fact_df=fact_df.drop(['Fatal_Cat','Fatal'], axis=1)
fact_df.head()

,Individual_Key,Special_Measure_Key,PHU_Key,Weather_Key,Resolved_no,Resolved_yes,Fatal_no,Fatal_yes
0,250,11,3895,123,0.0,1.0,1.0,0.0
1,322,11,3895,110,0.0,1.0,1.0,0.0
2,326,11,3895,96,1.0,0.0,0.0,1.0
3,347,11,3895,107,0.0,1.0,1.0,0.0
4,692,11,3895,116,0.0,1.0,1.0,0.0


### Now individuals table

In [10]:
indv_df = pd.DataFrame(individual_table, columns=['surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city'])
indv_df.head()

,surrogate_key,individual_id,date,age_group,gender,cause_of_infection,outbreak_related,reporting_phu_city
0,0,259,2020-03-25,50s,MALE,NO KNOWN EPI LINK,No,Toronto
1,1,5981,2020-03-16,20s,MALE,TRAVEL,No,Toronto
2,2,7345,2020-03-23,50s,MALE,OB,Yes,Toronto
3,3,7354,2020-03-24,20s,FEMALE,CC,No,Toronto
4,4,8096,2020-03-28,70s,FEMALE,TRAVEL,No,Ottawa


### Label Encoding for gender

In [11]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
indv_df['Gender_Cat'] = labelencoder.fit_transform(indv_df['gender'])
indv_df.head()

,surrogate_key,individual_id,date,age_group,gender,cause_of_infection,outbreak_related,reporting_phu_city,Gender_Cat
0,0,259,2020-03-25,50s,MALE,NO KNOWN EPI LINK,No,Toronto,2
1,1,5981,2020-03-16,20s,MALE,TRAVEL,No,Toronto,2
2,2,7345,2020-03-23,50s,MALE,OB,Yes,Toronto,2
3,3,7354,2020-03-24,20s,FEMALE,CC,No,Toronto,0
4,4,8096,2020-03-28,70s,FEMALE,TRAVEL,No,Ottawa,0


### One Hot Encoding for gender

In [12]:
# creating instance of one-hot-encoder

enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)

enc_df = pd.DataFrame(enc.fit_transform(indv_df[['Gender_Cat']]).toarray())

# merge with main df bridge_df on key values

indv_df = indv_df.join(enc_df)
indv_df.rename(columns={0: "Gender_female", 1: "Gender_gender_diverse", 2: "Gender_male", 3: "Gender_unspecified"},inplace = True)
indv_df=indv_df.drop(['Gender_Cat','gender'], axis=1)
indv_df.tail()

,surrogate_key,individual_id,date,age_group,cause_of_infection,outbreak_related,reporting_phu_city,Gender_female,Gender_gender_diverse,Gender_male,Gender_unspecified
89456,89456,289602,2021-01-29,30s,NO KNOWN EPI LINK,No,Toronto,1.0,0.0,0.0,0.0
89457,89457,289607,2021-01-29,40s,NO KNOWN EPI LINK,No,Toronto,1.0,0.0,0.0,0.0
89458,89458,289615,2021-02-02,30s,OB,Yes,Toronto,1.0,0.0,0.0,0.0
89459,89459,289620,2021-02-02,20s,NO KNOWN EPI LINK,No,Toronto,0.0,0.0,1.0,0.0
89460,89460,289621,2020-11-28,40s,MISSING INFORMATION,No,Toronto,0.0,0.0,1.0,0.0


In [13]:
#unspecified = indv_df[indv_df["gender"] == "UNSPECIFIED"]
#unspecified


In [14]:
#gend_div = indv_df[indv_df["gender"] == "GENDER DIVERSE"]
#gend_div